In [1]:
%load_ext autoreload
%autoreload 2
from qiskit.circuit.random import random_circuit as qiskit_random_circuit
from tnco.app import Optimizer, Tensor, TensorNetwork
import tnco.utils.tn as tn_utils
import more_itertools as mit
from random import Random
import quimb.tensor as qt
import itertools as its
import functools as fts
import operator as op
import numpy as np
import qiskit
import cirq

## Initialize Optimizer

In [2]:
# The optimizer can be initialized and reused multiple times. The optimization
# method can be specified using 'method' (the default method is simulated
# annealing, i.e. 'method=sa'). If 'max_width' is not provided, no memory
# constraints are enforced. If 'max_width' is provided, slices are added to the
# contraction to make sure that every intermediate tensor during the
# contraction will be within 'max_width' after the slices are applied.

# Initialize Optimizer (infinite memory)
opt = Optimizer()

# Initialize Optimizer (finite width)
opt_fw = Optimizer(max_width=2)

## Optimize `cirq.Circuit`

In [3]:
# 'tnco.app.Optimizer' accepts multiple format, including 'cirq.Circuit'.
# See: 'tnco.app.load_tn' for more informations.

# Get a random circuit
circuit = cirq.testing.random_circuit(qubits=8, n_moments=16, op_density=1)

# Get all qubits
qubits = sorted(circuit.all_qubits())

# The optimizer automatically simplify circuits and automatically decompose
# potential hyper-indices. To keep the final state simple without any
# hyper-index, let's add an initial / final layer of H**0.5, to avoid trivial
# simplifications.
circuit = (cirq.H**0.5).on_each(qubits) + circuit + (cirq.H**
                                                     0.5).on_each(qubits)

In [4]:
# The optimizer returns the tensor network used for the optimization, and the
# results of the optimization. If the provided tensor network is a circuit, the
# initial / final state can be provided. By default, the initial / final state
# are set to |0>...|0>. Otherwise, the initial / final state can be provided
# for each qubit as a 'dict'. If a qubit does not appear, it is considered as
# an open qubit. If initial / final state is None, all qubits are open.
# See 'tnco.app.load_tn' for more informations.

# Results assuming infinite memory
tn, res = opt.optimize(circuit,
                       betas=(0, 1e5),
                       initial_state='+',
                       final_state=None,
                       n_steps=1_000,
                       n_runs=4)

# Results enforcing finite width
tn_fw, res_fw = opt_fw.optimize(circuit,
                                betas=(0, 1e5),
                                initial_state='+',
                                final_state=None,
                                n_steps=1_000,
                                n_runs=4)

In [5]:
# ContractionResults contain useful informations regarding the optimal contraction:
# - .cost: The contraction cost in floating-point-operations (FLOP)
# - .path: The contraction path
# - .runtime_s: The total runtime spent for the optimization
#
# In case the tensor network to optimize is composed of disconnected components,
# the cost / path for each connected component is also provided:
# - .disconnected_costs: The contraction cost in FLOP for each connected
#                        component
# - .disconnected_paths: The contraction path for each connected component
#
# If 'max_width' is provided, the cost would reflect the presence of slices.
# ContractionResults will also include the sliced indices:
# - .slices: Sliced indices
# - .disconnected_sliced: Sliced indices for each connected component

# Sort accordingly to the cost
res = sorted(res, key=lambda x: x.cost)
res_fw = sorted(res_fw, key=lambda x: x.cost)

# Print stats
print('# Infinite Memory| Runtime: {:1.2g}s'.format(
    sum(map(lambda x: x.runtime_s, res)) / len(res)))
print('# Infinite Memory| log10(FLOP): {:1.2g}'.format(res[0].cost.log10()))
print('# -')
print('# Finite Width| Runtime: {:1.2g}s'.format(
    sum(map(lambda x: x.runtime_s, res)) / len(res)))
print('# Finite Width| log10(FLOP): {:1.2g}'.format(res_fw[0].cost.log10()))
print('# Finite Width| Number of sliced indices: {}'.format(
    len(res_fw[0].slices)))

# Get contraction path (infinite memory)
path = res[0].path

# Get contraction path (finite width)
path_fw = res_fw[0].path

# Infinite Memory| Runtime: 0.0044s
# Infinite Memory| log10(FLOP): 3.7
# -
# Finite Width| Runtime: 0.0044s
# Finite Width| log10(FLOP): 6.1
# Finite Width| Number of sliced indices: 14


In [6]:
# The resulting contraction can be contracted using your favorite library.
# Ouput indices in the tensor network are named '(qubit_name, i)', with
# 'qubit_name' being the name of the qubit, and 'i' is zero only if 'qubit_name'
# belongs to the initial state.

# Get the exact final state
exact_final_state = cirq.Simulator().simulate(
    circuit,
    initial_state=np.ones(2**len(qubits)) / np.sqrt(2**len(qubits)),
    qubit_order=qubits).state_vector()

# Get the final state
final_state = qt.TensorNetwork(map(qt.Tensor, tn.arrays, tn.ts_inds)).contract(
    optimize=path, output_inds=tn.output_inds)

# No qubit should belong to the initial state
assert all(map(lambda x: x != 0, map(op.itemgetter(1), final_state.inds)))

# Re-index output inds for a quicker access
final_state = final_state.reindex(dict(
    zip(final_state.inds, map(op.itemgetter(0), final_state.inds))),
                                  inplace=True)

# Transpose final state accordingly to qubits
final_state = final_state.transpose(*qubits, inplace=True)

# Check
np.testing.assert_allclose(final_state.data.ravel(),
                           exact_final_state,
                           atol=1e-5)

## Optimize `qiskit.QuantumCircuit`

In [7]:
# Similarly, the optimizer can use 'qiskit.QuantumCircuit' as input

# Implement sqrt of Hadamard
sqrt_H = qiskit.circuit.library.UnitaryGate(cirq.unitary(cirq.H**0.5),
                                            label='√H')

# Get a random circuit
qiskit_circuit = qiskit.QuantumCircuit(8)
mit.consume(map(lambda i: qiskit_circuit.append(sqrt_H, [i]), range(8)))
qiskit_circuit = qiskit_circuit.compose(qiskit_random_circuit(8, 16))
mit.consume(map(lambda i: qiskit_circuit.append(sqrt_H, [i]), range(8)))

# Optimize circuit
tn, res = opt.optimize(qiskit_circuit,
                       betas=(0, 1e5),
                       initial_state='+',
                       final_state=None,
                       n_steps=1_000,
                       n_runs=4)

## Optimize Gates

In [8]:
# Instead of using 'cirq.Circuit', it is possible to directly provide a list of
# matrices with their indices.
gates = list(
    map(lambda gate: (cirq.unitary(gate), gate.qubits),
        circuit.all_operations()))

# Optimize the tensor network
tn, res = opt.optimize(gates,
                       betas=(0, 1e5),
                       initial_state='+',
                       final_state=None,
                       n_steps=1_000,
                       n_runs=4)

# Get path
path = sorted(res, key=lambda x: x.cost)[0].path

# Get the final state
final_state = qt.TensorNetwork(map(qt.Tensor, tn.arrays, tn.ts_inds)).contract(
    optimize=path, output_inds=tn.output_inds)

# No qubit should belong to the initial state
assert all(map(lambda x: x != 0, map(op.itemgetter(1), final_state.inds)))

# Re-index output inds for a quicker access
final_state = final_state.reindex(dict(
    zip(final_state.inds, map(op.itemgetter(0), final_state.inds))),
                                  inplace=True)

# Transpose final state accordingly to qubits
final_state = final_state.transpose(*qubits, inplace=True)

# Check
np.testing.assert_allclose(final_state.data.ravel(),
                           exact_final_state,
                           atol=1e-5)

## Optimize Arbitrary Tensor Network

In [9]:
# An arbitrary tensor tensor can be specified by using 'tnco.app.TensorNetwork'
from tnco.tests.utils import generate_random_tensors

# Get a list of random indices for each tensor. The resulting tensor network
# will have two disconnected components, 2 * 10 tensors (10 tensors for each
# connected component), 2 * 3 output indices (2 indices for each connected
# component),  and every index will be a hyper-index of degree 3.
ts_inds, output_inds = generate_random_tensors(n_tensors=10,
                                               n_inds=20,
                                               n_cc=2,
                                               k=3,
                                               n_output_inds=3)

# Get all indices
inds = frozenset(mit.flatten(ts_inds))

# Get some random dimensions
dims = dict(zip(inds, Random().choices(range(2, 4), k=len(inds))))

# There should be 2 * 10 tensors
assert len(ts_inds) == 2 * 10

# There should be 2 * 20 indices
assert len(inds) == 2 * 20

# There should be 2 * 3 open indices
assert len(output_inds) == 2 * 3

# Get tensor network. For the optimization, arrays are not required
tn = TensorNetwork(map(
    lambda id, xs: Tensor(xs, map(dims.get, xs), tags=dict(id=id)), its.count(),
    ts_inds),
                   output_inds=output_inds)

# Optimize the tensor network. Because the tensor network is simplified before
# being optimized, the output 'tn' might be different from the input 'tn'. Also,
# if arrays are not provided, the decomposition of hyper indices is
# automatically disabled
new_tn, res = opt.optimize(tn, fuse=10, betas=(0, 1e5), n_steps=1_000, n_runs=4)

/usr/local/google/home/smandra/projects/tnco_v2_clean/tnco/app/app.py:318: UserWarning: Cannot decompose hyper-indices if not all arrays are provided.
  warn("Cannot decompose hyper-indices if not "


In [10]:
# Given the optimized path, we can later contract the actual tensors
arrays = list(map(lambda t: np.random.normal(size=t.dims), tn.tensors))

# Fused tensors are kept track in 'TensorNetwork.tags['fuse_path']'
ts_inds, output_inds, new_arrays = tn_utils.contract(new_tn.tags['fuse_path'],
                                                     ts_inds=tn.ts_inds,
                                                     output_inds=tn.output_inds,
                                                     dims=tn.dims,
                                                     arrays=arrays)

# Check consistency with new_tn
assert new_tn.ts_inds == tuple(ts_inds)
assert new_tn.output_inds == output_inds
assert tn.output_inds == new_tn.output_inds

# Contract the arrays before the optimization
final_tensor = qt.TensorNetwork(map(
    qt.Tensor, arrays, tn.ts_inds)).contract(output_inds=tn.output_inds)

# Contract the arrays after the optimization
new_final_tensor = qt.TensorNetwork(map(
    qt.Tensor, new_arrays, new_tn.ts_inds)).contract(
        optimize=sorted(res, key=lambda x: x.cost)[0].path,
        output_inds=new_tn.output_inds).transpose_like(final_tensor)

# Check
np.testing.assert_allclose(final_tensor.data, new_final_tensor.data, atol=1e-5)

In [11]:
# The optimizer also accepts a list of indices of the format:
#
# [
#  (dim_1, tensor_1_1, tensor_1_2, ... tensor_1_k1),
#  ...
#  (dim_n, tensor_n_1, tensor_n_2, ... tensor_n_kn)
# ]
#
# Open indices can be specified using the token (by default: '*'):
#
# [
#  (dim_1, tensor_1_1, tensor_1_2, ... tensor_1_k1, '*'),
#  ...
#  (dim_n, tensor_n_1, tensor_n_2, ... tensor_n_kn)
# ]
#
# See 'tnco.app.load_tn' and 'tnco.utils.tn.read_inds' for more informations.

In [12]:
# Create a map of indices. To track the location in 'arrays' of the tensors,
# let's enumerate them and use it as name of the tensor.
inds_map = dict(zip(dims, map(lambda d: [d], dims.values())))
for idx, tensor in enumerate(tn.tensors):
    for x in tensor.inds:
        inds_map[x].append(idx)

# Add token to output indices
output_index_token = '*'
for x in tn.output_inds:
    inds_map[x].append(output_index_token)

# Optimize the tensor network
new_tn, res = opt.optimize(inds_map.values(),
                           betas=(0, 1e5),
                           n_steps=1_000,
                           output_index_token=output_index_token,
                           n_runs=4)

# Because indices don't have any name, they are named using the order in which
# they appear in 'inds_map'
assert frozenset(map(fts.partial(mit.nth, inds_map),
                     new_tn.output_inds)) == tn.output_inds

In [13]:
# An easier way would be to load the tensor network first, and then optimize
# it. Because we used a list of indices, the order of the tensors and the
# indices might have changed.
new_tn_map, new_dims, new_output_inds, new_sparse_inds = tn_utils.read_inds(
    inds_map, output_index_token=output_index_token)
new_ts_inds = list(map(op.itemgetter(1), sorted(new_tn_map.items())))

# Get the new arrays
new_arrays = list(
    map(lambda a, old_xs, new_xs: qt.Tensor(a, old_xs).transpose(*new_xs).data,
        arrays, tn.ts_inds, new_ts_inds))

# Initialize the new tensor network to optimize
new_tn = TensorNetwork(map(lambda xs, a: Tensor(xs, array=a), new_ts_inds,
                           new_arrays),
                       output_inds=new_output_inds)

# Check consistency
assert all(
    map(lambda xs, ys: frozenset(xs) == frozenset(ys), tn.ts_inds,
        new_tn.ts_inds))
assert tn.output_inds == new_tn.output_inds
assert tn.sparse_inds == new_tn.sparse_inds
assert tn.dims == new_tn.dims

# Optimize it
new_tn, res = opt.optimize(new_tn,
                           fuse=10,
                           betas=(0, 1e5),
                           n_steps=1_000,
                           n_runs=4)

# Contract the tensor network using the optimized path
new_final_tensor = qt.TensorNetwork(
    map(qt.Tensor, new_tn.arrays,
        new_tn.ts_inds)).contract(optimize=sorted(res,
                                                  key=lambda x: x.cost)[0].path,
                                  output_inds=new_tn.output_inds)

# Because indices have been decomposed in hyper-inds, the output indices might be
# different. We can re-index them using the hyper-indices decomposition map
new_final_tensor = new_final_tensor.reindex(dict(
    zip(map(new_tn.tags['hyper_inds_map'].get, tn.output_inds),
        tn.output_inds)),
                                            inplace=True)

# Also, it might happen that a different decomposition of hyper indices lead to
# the removal of one or more output indices. It happens, for instance, if only
# one projection for the removed indices is different from zero.
assert frozenset(new_final_tensor.inds).issubset(
    final_tensor.inds) or frozenset(final_tensor.inds).issubset(
        new_final_tensor.inds)

# Transpose
new_final_tensor = new_final_tensor.transpose_like(final_tensor, inplace=True)

# Check
np.testing.assert_allclose(final_tensor.data, new_final_tensor.data, atol=1e-5)